In [ ]:
########################
###### Changerate ######
########################

# Imports
import pymongo, urllib, json, datetime, psycopg2
from pymongo import MongoClient
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from functools import reduce
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Connect to MongoDB
pw = ""
cluster = MongoClient('mongodb+srv:')
db = cluster["trends"]
collection_berlin = db["berlin"]

# Get trends from MongoDB
trends_berlin = collection_berlin.find()

# Close connection to MongoDB
cluster.close()

# If not already created automatically, instantiate Sparkcontext
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

# Connect to Postgres
conn = psycopg2.connect(host="", port = 5432, database="trends", user="", password="")

# Get or create Spark Context and Spark Session
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

# Create empty list for dataframes
df_list_trends_berlin = list()

# Iterate over list of trends
counter_trends = 1
for trends in trends_berlin:
    
    # Create PySpark dataframe
    df_trends = spark.createDataFrame([], StringType())

    # Iterate over trends at listelement
    counter = 0
    while counter <= 49:
        
        # Append to list of dataframes
        newRow = spark.createDataFrame([trends["trends"][counter]["name"].replace("#","")], StringType())
        df_trends = df_trends.union(newRow)
        counter+=1
        
    # Append to list of dataframes
    df_list_trends_berlin.append(df_trends)
    
    # Every second
    if counter_trends == 2:
        
        # Join dataframes of trends by equal values
        df_equality = df_list_trends_berlin[0].join(df_list_trends_berlin[1], [df_list_trends_berlin[0].value == df_list_trends_berlin[1].value] , how = 'inner' )
        
        # Calculate changerate by subtract equality from 100%
        equality = df_equality.count()/50
        changerate = round(1 - equality, 2)
        changerate = changerate * 100
        
        # Get datetime of trends
        date = trends["as_of"].replace("T"," ").replace("Z","")
        date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        date = date + datetime.timedelta(hours=2)

        # Insert data into PostgreSQL
        cur = conn.cursor()
        cur.execute("""INSERT INTO Changerate (changerate, date) VALUES ("""+str(changerate)+""",'"""+str(date)+"""')""")
        conn.commit()
        cur.close()
        
        # Delete first element
        del df_list_trends_berlin[0]
        counter_trends = 1
        
        # Print results
        print("Changerate: "+str(changerate)+"")
        print("Datetime: "+str(date)+"")
        print("""INSERT INTO Changerate (changerate, date) VALUES ("""+str(changerate)+""",'"""+str(date)+"""')""")
        print("----------------------------------------------------------------------------------")
        
    counter_trends+=1

# Close connection to DB and Spark
conn.close()
spark.stop()